In [1]:
import os

In [2]:
%pwd

'c:\\Users\\hp\\end-to-end-text-summarization\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'c:\\Users\\hp\\end-to-end-text-summarization'

In [1]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class ModelTrainerConfig:
    root_dir: Path
    data_path: Path
    model_ckpt: Path
    num_train_epochs: int
    warmup_steps: int
    per_device_train_batch_size: int
    weight_decay: float
    logging_steps: int
    evaluation_strategy: int
    eval_steps: int
    save_steps: int
    gradient_accumulation_steps: int

In [2]:
from textsummarizer.constants import *
from textsummarizer.utils.common import read_yaml, create_directories

In [3]:
class configurationManager:
    def __init__(
            self,
            config_filepath = CONFIG_FILE_PATH,
            params_filepath = PARAMS_FILE_PATH):
        
            self.config = read_yaml(config_filepath)
            self.params = read_yaml(params_filepath)

            create_directories([self.config.artifacts_root])

    def get_model_trainer_config(self) -> ModelTrainerConfig:
        config = self.config.model_trainer
        params = self.params.TrainingArguments

        create_directories([config.root_dir])

        model_trainer_config = ModelTrainerConfig(
            root_dir = config.root_dir,
            data_path = config.data_path,
            model_ckpt = config.model_ckpt,
            num_train_epochs = params.num_train_epochs,
            warmup_steps = params.warmup_steps,
            per_device_train_batch_size  = params.per_device_train_batch_size,
            weight_decay = params.weight_decay,
            logging_steps = params.logging_steps,
            evaluation_strategy = params.evaluation_strategy,
            eval_steps = params.eval_steps,
            save_steps = params.save_steps,
            gradient_accumulation_steps = params.gradient_accumulation_steps
            )
        
        return model_trainer_config



In [5]:
import os
from transformers import TrainingArguments, Trainer
from transformers import DataCollatorForSeq2Seq
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
from transformers import AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq
import torch
from datasets import load_dataset, load_from_disk

In [6]:
class ModelTrainer:
    def __init__(self,config: ModelTrainerConfig):
        self.config = config

    def train(self):
        device = "cuda" if torch.cuda.is_available() else "cpu"
        tokenizer = AutoTokenizer.from_pretrained(self.config.model_ckpt)
        model_pegasus = AutoModelForSeq2SeqLM.from_pretrained(self.config.model_ckpt).to(device)
        seq2seq_data_collector = DataCollatorForSeq2Seq(tokenizer,model=model_pegasus)

        # loading data
        dataset_sansum_pt = load_from_disk(self.config.data_path)

       # trainer_args = TrainingArguments(
            #output_dir = self.config.root_dir,
            #data_path = self.config.data_path,
            #model_ckpt = self.config.model_ckpt,
            #num_train_epochs = self.params.num_train_epochs,
            #warmup_steps = self.params.warmup_steps,
            #per_device_train_batch_size  = self.params.per_device_train_batch_size,
            #weight_decay = self.params.weight_decay,
            #logging_steps = self.params.logging_steps,
            #evaluation_strategy = self.params.evaluation_strategy,
            #eval_steps = self.params.eval_steps = 1e6,
            #save_steps = self.params.save_steps,
            #gradient_accumulation_steps = self.params.gradient_accumulation_steps
        #)

        trainer_args = TrainingArguments(
            output_dir = self.config.root_dir, num_train_epochs=1,warmup_steps=500,
            per_device_train_batch_size=1, per_device_eval_batch_size=1,
            weight_decay=0.01, logging_steps=10,
            evaluation_strategy='steps',eval_steps=500, save_steps=1e6,
            gradient_accumulation_steps=16
)
        
        trainer = Trainer(model=model_pegasus, args = trainer_args,
                  tokenizer=tokenizer,data_collator=seq2seq_data_collector,
                  train_dataset=dataset_sansum_pt["test"],
                  eval_dataset=dataset_sansum_pt["validation"])
        
        trainer.train()

        # save model
        model_pegasus.save_model(os.path.join(self.config.root_dir,"pegasus-samsum-model"))

        # save tokenizer
        tokenizer.save_pretrained(os.path.join(self.config.root_dir,"tokenizer"))

In [9]:
try:
    config = configurationManager()
    model_trainer_config = config.get_model_trainer_config()
    model_trainer_config = ModelTrainer(config=model_trainer_config)
    model_trainer_config.train()
except Exception as e:
    raise e

FileNotFoundError: [Errno 2] No such file or directory: 'config\\config.yaml'